In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")

In [ ]:
# How to create a collection with two named vectors
from weaviate.classes.config import Configure, Property, DataType

client.collections.create(
    "DocumentNamedVectors",
     # Define properties
    properties=[
        Property(name="summary", data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT)
    ],
    vector_config=[
        # Set a named vector
        Configure.Vectors.text2vec_openai(
            name="SummaryNamedVector",
            source_properties=["summary"]       
        ),
        # Set another named vector
        Configure.Vectors.text2vec_weaviate(
            name="ContentNamedVector",
            source_properties=["content"],
            model="Snowflake/snowflake-arctic-embed-m-v1.5",
            vectorize_collection_name=False
        )
    ]
)


In [ ]:
# Get the collection "DocumentNamedVectors"
coll = client.collections.get("DocumentNamedVectors")

# Define 10 objects with meaningful dummy data
dummy_data = [
    {
        "summary": "Quantum Computing Breakthrough",
        "content": "A breakthrough in quantum computing has accelerated data processing speeds across multiple tech industries."
    },
    {
        "summary": "Global Economic Update",
        "content": "Several countries report fluctuations in GDP growth as global markets react to recent policy changes."
    },
    {
        "summary": "Advances in Healthcare",
        "content": "New clinical trials have shown promising results in the fight against chronic diseases with innovative treatments."
    },
    {
        "summary": "Renewable Energy Push",
        "content": "Governments worldwide are increasing investments in renewable energy sources to combat climate change."
    },
    {
        "summary": "Revolution in Education",
        "content": "Innovative e-learning platforms and digital classrooms are transforming the traditional education system."
    },
    {
        "summary": "Sports Championship Highlights",
        "content": "The national team secured a thrilling victory in the championship final, uniting fans across the country."
    },
    {
        "summary": "AI Transforming Industries",
        "content": "Artificial intelligence applications are innovating sectors such as manufacturing, logistics, and healthcare."
    },
    {
        "summary": "Wildlife Conservation Success",
        "content": "Conservationists celebrate a major victory as endangered species see a resurgence due to new protection laws."
    },
    {
        "summary": "Space Exploration Milestone",
        "content": "A private space company successfully landed its spacecraft on Mars, marking a milestone in interplanetary travel."
    },
    {
        "summary": "Retail Industry Innovations",
        "content": "Retailers are embracing digital transformation with cutting-edge online platforms and enhanced customer experiences."
    }
]

# Insert each object into the collection and print the resulting UUID
for data in dummy_data:
    uuid = coll.data.insert(data)
    print(f"Inserted object with UUID: {uuid}")

In [ ]:
# Fetch the object with the named vectors
collection = client.collections.get("<COLLECTION_NAME>")
response = collection.query.fetch_objects()
vector_names = ["<NAMED_VECTOR_PROP>", "NAMED_VECTOR_PROP"]

data_object = collection.query.fetch_object_by_id(
    uuid="<UUID>", 
    include_vector=vector_names
)

print(data_object)

In [ ]:
# Querying with named vectors
from weaviate.classes.query import MetadataQuery

col = client.collections.get("<COLLECTION_NAME>")
response = col.query.near_text(
    query="<QUERY>",
    target_vector="NAMED_VECTOR_PROP",
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

In [ ]:
# Filter with named vectors example
from weaviate.classes.query import Filter

col = client.collections.get("DocumentNamedVectors")
response = col.query.hybrid(
    query="healthcare",
    target_vector="SummaryNamedVector",
    filters=Filter.by_property("summary").like("healthcare")
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

In [ ]:
# Get the collection and check if the collection has named vectors
collection = client.collections.get("DocumentNamedVectors")
config_data = collection.config.get()

# Check if collection has named vectors
if config_data.vector_config is not None and len(config_data.vector_config) > 0:
    print(f"Collection '{config_data.name}' has named vectors:")
    for vector_name in config_data.vector_config.keys():
        print(f"  - {vector_name}")
else:
    print(f"Collection '{config_data.name}' uses default vectorizer (no named vectors)")